# 功能合併

In [4]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
# ------------------------------------------------- modified for captcha
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# ------------------------------------------------- modified for captcha

def to_df():
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.find_all('tr')[1:]
    rows = list()
    for tr in trs:
        rows.append([td.text.replace('\n', '').replace('\xa0', '').replace('\t', '') for td in tr.find_all('td')])
    df = pd.DataFrame(data=rows)
    return df    

def grab_GPA(semester):
    # 選所有課程
    Select(driver.find_element(by=By.ID, value="qyt_id")).select_by_visible_text(semester)
    driver.find_element(by=By.NAME, value="Submit").click()

    # 按掉alert
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    driver.switch_to.alert.accept()

    # 等所有資料load
    sleep(10)
    
    # 表格轉dataframe
    df=to_df()
    df.columns=['科號 Course No.','科目名稱 Course Name','授課教師 Teacher','修課人數 Enrollment',
                '平均值 average GPA','標準差 standard deviation']
    #上一頁
    driver.find_element(by=By.NAME, value="Submit").click()
    
    return df
    
# ------------------------------------------------- modified for Brady's computer
options = Options()
options.add_argument("--disable-notifications")
options.add_experimental_option('excludeSwitches', ['enable-logging']) # shut up error code triggered by chrome: "Failed to read descriptor from node connection: A device attached to the system is not functioning." 

# win chrome
driver_path = './chromedriver_win'
s = Service(driver_path)
driver = webdriver.Chrome(service= s, options= options)
# ------------------------------------------------- modified for Brady's computer

# # ------ original
# driver = webdriver.Chrome()
# # ------ original
verificationErrors = []
accept_next_alert = True

driver.get("https://www.ccxp.nthu.edu.tw/ccxp/INQUIRE/")

driver.find_element(by=By.NAME, value="account").clear()
driver.find_element(by=By.NAME, value="account").send_keys("108060071") #要改成自己的帳號
driver.find_element(by=By.NAME, value="passwd").clear()
driver.find_element(by=By.NAME, value="passwd").send_keys("Hanna891210") #要改成自己的密碼
driver.find_element(by=By.XPATH, value=u"(.//*[normalize-space(text()) and normalize-space(.)='密 碼:'])[1]/following::tr[1]").click()
driver.find_element(by=By.NAME, value="passwd2").click()
driver.find_element(by=By.NAME, value="passwd2").clear()

# ------------------------------------------------- modified for captcha
from captcha import captcha_decoder
image_url = driver.find_element(By.XPATH, "//input[@name='passwd2']/following-sibling::img").get_attribute("src")
captcha_result = captcha_decoder(image_url)
captcha = driver.find_element(By.NAME, "passwd2")
captcha.send_keys(captcha_result)
# ------------------------------------------------- modified for captcha
# # ------ 手動輸入 驗證碼
# sleep(5)
# # ------ 手動輸入 驗證碼
driver.find_element(by=By.NAME, value="Submit").click()



In [ ]:
# 切換到修課同學
driver.switch_to.frame(1)
driver.find_element(by=By.ID, value="nodeIcon15").click() #課程、成績
driver.find_element(by=By.ID, value="itemIcon21").click() #修課同學查詢

# 切到中間的畫面
driver.switch_to.parent_frame()
driver.switch_to.frame(2)

GPA_110_1=grab_GPA("110上(2021-Fall)")
GPA_109_1=grab_GPA("109上(2020-Fall)") 

# 切到左邊側攔
driver.switch_to.parent_frame()
driver.switch_to.frame(1)
driver.find_element(by=By.ID, value="nodeIcon54").click() # 選課
driver.find_element(by=By.ID, value="nodeIcon61").click() # 報表
driver.find_element(by=By.ID, value="itemIcon62").click() # 選上/剩餘名額/待亂數人數統計

# 切到中間的畫面
driver.switch_to.parent_frame()
driver.switch_to.frame(2)

# 選通識中心課程
Select(driver.find_element(by=By.NAME, value="select")).select_by_visible_text('GE 通識中心')
driver.find_element(by=By.NAME, value="Submit").click()

# 等資料load
sleep(5)

# 表格轉dataframe
enrollment_status_df=to_df()
enrollment_status_df.columns=['科號 Course Number.','科目名稱 Course Name','教師 Professor','通識類別',
                              '時間 Class Time','人數限制 / 新生保留 Size Limit / Freshman only','目前選上人數 Current Number',
                              '目前剩餘名額 Current Spots Remaining','目前待亂數人數 To be randomed']
enrollment_status_df=enrollment_status_df.drop([0])
enrollment_status_df=enrollment_status_df.drop([1])

driver.quit()

## 選上機率

In [59]:
filtered_df['目前剩餘名額 Current Spots Remaining'] = pd.to_numeric(filtered_df['目前剩餘名額 Current Spots Remaining'])
filtered_df['目前待亂數人數 To be randomed'] = pd.to_numeric(filtered_df['目前待亂數人數 To be randomed'])

c1 =[]
for index, row in filtered_df.iterrows():
    if(row['目前剩餘名額 Current Spots Remaining'] == 0):
        c1.append(0)
    elif(row['目前剩餘名額 Current Spots Remaining'] != 0 and row['目前待亂數人數 To be randomed'] == 0):
        c1.append(100)
    elif(row['目前剩餘名額 Current Spots Remaining'] > row['目前待亂數人數 To be randomed']):
        c1.append(100)
    else:
        c1.append((row['目前剩餘名額 Current Spots Remaining'] / row['目前待亂數人數 To be randomed'])*100)

filtered_df['選上機率'] = c1